In [34]:
import QuantLib as ql
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from random import sample
import sklearn.neural_network as sknn
import matplotlib.pyplot as plt
import time
import math

In [36]:
def MakeUniformRandParams(ArrayMin,ArrayMax,nRand):
    if (len(ArrayMin)!=len(ArrayMax)):
        raise Exception('ArrayMax and ArrayMin must have the same size')
    else:
        nParams = len(ArrayMin)
        rand_seq = ql.UniformRandomSequenceGenerator(nParams*nRand,ql.UniformRandomGenerator())
        X = rand_seq.nextSequence().value()
        #X = [ArrayMin[j%nParams]+(ArrayMax[j%nParams]-ArrayMin[j%nParams])*X[j] for j in range(nParams*nRand)]
        res = np.zeros((nRand,nParams))
        LocalMax = 0
        res__ = 0
        res_ = 0
        for i in range(nRand):
            for j in range(nParams):
                AdjMax = ArrayMax[j]
                if j==2:
                    AdjMax = XiFellerAdjust(res__,res_,ArrayMax[j])
                res[i,j]=ArrayMin[j]+(AdjMax-ArrayMin[j])*X[i*nParams+j]
                res__ = res_
                res_ = res[i,j]
        #res = np.reshape(X,(nRand,nParams))
        return res

def CountNoFellerCondition(X):
    k=0
    for j in range(len(X)):
        if (X[j,2]>XiFellerMax(X[j,0],X[j,1])):
            k=k+1
    return k

def XiFellerMax(kappa,theta):
    return math.sqrt(2*kappa*theta)

def XiFellerAdjust(kappa,theta,xi_init):
    if (xi_init>XiFellerMax(kappa,theta)):
        return XiFellerMax(kappa,theta)
    return xi_init
    
ArrayMin = [0.1, 0.1,  0.01, -0.999, 0.1]
ArrayMax = [1.5, 1.5,  0.75,  0.999, 1.5]
nRand = 100000
start = time.time()
res=MakeUniformRandParams(ArrayMin,ArrayMax,nRand)
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')
print('No Feller Condition ' + str(CountNoFellerCondition(res)))

Total computation time = 0.45s
No Feller Condition 0


In [37]:
# GLOBAL CONFIG #
AsOfDate = ql.Date(15,10,2018)
ql.Settings.instance().evaluationDate = AsOfDate
DayCount = ql.Actual365Fixed()
Calendar = ql.UnitedStates()

In [71]:
MaturityDates = []
strikes = []
exercises = []
payoffs = []
for k in range(8):
    MaturityDates.append(ql.Date(15,10,2018+k+1))
    exercises.append(ql.EuropeanExercise(MaturityDates[k]))
for k in range(11):
    strikes.append(50+k*10)
    payoffs.append(ql.PlainVanillaPayoff(option_type, strikes[k]))
print(MaturityDates)
print(strikes)
option_type = ql.Option.Call
EuropeanOptions = []
for T in range(len(MaturityDates)):
    for K in range(len(strikes)):
        EuropeanOptions.append(ql.VanillaOption(payoffs[K], exercises[T]))
#print(EuropeanOptions)

[Date(15,10,2019), Date(15,10,2020), Date(15,10,2021), Date(15,10,2022), Date(15,10,2023), Date(15,10,2024), Date(15,10,2025), Date(15,10,2026)]
[50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]


In [73]:
# EUROPEAN OPTION CONFIG #
MaturityDate = ql.Date(15,10,2022)
strike = 100
option_type = ql.Option.Call
payoff = ql.PlainVanillaPayoff(option_type, strike)
exercise = ql.EuropeanExercise(MaturityDate)
EuropeanOption = ql.VanillaOption(payoff, exercise)

In [74]:
# MARKET DATA
r=ql.SimpleQuote(0.01)
r_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(r),DayCount)
q=ql.SimpleQuote(0.0)
q_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(q),DayCount)
s0=ql.SimpleQuote(100)

# BLACK SCHOLES ENGINE ONLY USED FOR COMPUTING IMPLIED VOLATILITIES
vol_bs = ql.SimpleQuote(0.10)
vol_bs_ts = ql.BlackConstantVol(0,Calendar,ql.QuoteHandle(vol_bs),DayCount)
BSProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(s0),ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                               ql.BlackVolTermStructureHandle(vol_bs_ts))

# HESTON ENGINE
v0 = 0.01
kappa=0.5
theta=0.01
sigma=0.0001
rho=-0.5
HestonParams = [theta,kappa,sigma,rho,v0]
relTolerance=0.01
maxEval=20000
HestonProcess = ql.HestonProcess(ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                                 ql.QuoteHandle(s0),v0,kappa,theta,sigma,rho)
HestonModel = ql.HestonModel(HestonProcess)
HestonEngine = ql.AnalyticHestonEngine(HestonModel,relTolerance,maxEval)
EuropeanOption.setPricingEngine(HestonEngine)
for i in range(len(EuropeanOptions)):
    EuropeanOptions[i].setPricingEngine(HestonEngine)

In [84]:
# GENERATE DATAS
nRand = 150
start = time.time()
MinParamsValues = [0.1, 0.1,  0.01, -0.999, 0.1]
MaxParamsValues = [1.5, 1.5,  0.75,  0.999, 1.5]
local_start = 0
local_end = 0
ArrParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)
intermediary = time.time()

#npv = np.zeros(nRand)
#iv = np.zeros(nRand)
#for i in range(nRand):
#    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
#    npv[i] = EuropeanOption.NPV()
#    iv[i] = EuropeanOption.impliedVolatility(npv[i],BSProcess,1.0e-4,200,1.0e-8,10)
#end = time.time()
#print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
#print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
#print('Total computation time = ' + str(round(end-start,2)) + 's')

npv = np.zeros(nRand*len(EuropeanOptions))
iv = np.zeros(nRand*len(EuropeanOptions))
for i in range(nRand):
    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
    for j in range(len(EuropeanOptions)):
        npv[i*len(EuropeanOptions)+j] = EuropeanOptions[j].NPV()
        print(npv[i*len(EuropeanOptions)+j])
        print('K = ' + str(strikes[j%len(strikes)]))
        print('T = ' + str(MaturityDates[int(j/len(strikes))]))
        iv[i*len(EuropeanOptions)+j] = EuropeanOptions[j].impliedVolatility(npv[i*len(EuropeanOptions)+j],BSProcess,1.0e-4,200,1.0e-8,100)
end = time.time()
print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
print('Total computation time = ' + str(round(end-start,2)) + 's')



53.277191619439165
K = 50
T = October 15th, 2019
46.71509414269843
K = 60
T = October 15th, 2019
41.304728712604465
K = 70
T = October 15th, 2019
36.82139227201811
K = 80
T = October 15th, 2019
33.0734843580074
K = 90
T = October 15th, 2019
29.910025907047782
K = 100
T = October 15th, 2019
27.214644696812073
K = 110
T = October 15th, 2019
24.897197494989406
K = 120
T = October 15th, 2019
22.8884127919026
K = 130
T = October 15th, 2019
21.13446000790005
K = 140
T = October 15th, 2019
19.592310868614977
K = 150
T = October 15th, 2019
59.740654801116776
K = 50
T = October 15th, 2020
55.286832601246644
K = 60
T = October 15th, 2020
51.58793992931135
K = 70
T = October 15th, 2020
48.459699744633824
K = 80
T = October 15th, 2020
45.7726041193148
K = 90
T = October 15th, 2020
43.43423247542707
K = 100
T = October 15th, 2020
41.37684333952018
K = 110
T = October 15th, 2020
39.54923226057734
K = 120
T = October 15th, 2020
37.91213678340585
K = 130
T = October 15th, 2020
36.43523972332527
K = 14

T = October 15th, 2024
69.8512080704689
K = 90
T = October 15th, 2024
68.3339885193842
K = 100
T = October 15th, 2024
66.9383210188892
K = 110
T = October 15th, 2024
65.6466805264284
K = 120
T = October 15th, 2024
64.44511339976896
K = 130
T = October 15th, 2024
63.322318554613254
K = 140
T = October 15th, 2024
62.26900874940658
K = 150
T = October 15th, 2024
80.91684531202921
K = 50
T = October 15th, 2025
78.98194247245566
K = 60
T = October 15th, 2025
77.35756005843814
K = 70
T = October 15th, 2025
75.7256933694105
K = 80
T = October 15th, 2025
74.32551279336009
K = 90
T = October 15th, 2025
73.04240473555042
K = 100
T = October 15th, 2025
71.85814483202877
K = 110
T = October 15th, 2025
70.75854430959278
K = 120
T = October 15th, 2025
69.73232251992613
K = 130
T = October 15th, 2025
68.7703516786064
K = 140
T = October 15th, 2025
67.86513435666346
K = 150
T = October 15th, 2025
83.65159172604905
K = 50
T = October 15th, 2026
82.02743608405395
K = 60
T = October 15th, 2026
80.6789023

RuntimeError: root not bracketed: f[1e-08,100] -> [-5.082141e+01,-8.214113e-01]

In [48]:
def ann_layers_neurones(layer, neurone):
    hidden_layer_sizes = (neurone, )
    if layer == 1.0: return hidden_layer_sizes
    elif layer > 1.0:
        for i in range(layer-1):
            hidden_layer_sizes += (neurone,)
        return hidden_layer_sizes

nLayersMin = 3
nLayersMax = 5
nLayersStep = 1
nNeuronesMin = 30
nNeuronesMax = 60
nNeuronesStep = 5
hidden_layers_sizes_vect = []
i_ = 0
for i in range(nNeuronesMin,nNeuronesMax+1,nNeuronesStep):
    j_ = 0
    for j in range(nLayersMin,nLayersMax+1,nLayersStep):
        hidden_layers_sizes_vect.append(ann_layers_neurones(j,i))
        j_ += 1
    i_ += 1
print(hidden_layers_sizes_vect)

[(30, 30, 30), (30, 30, 30, 30), (30, 30, 30, 30, 30), (35, 35, 35), (35, 35, 35, 35), (35, 35, 35, 35, 35), (40, 40, 40), (40, 40, 40, 40), (40, 40, 40, 40, 40), (45, 45, 45), (45, 45, 45, 45), (45, 45, 45, 45, 45), (50, 50, 50), (50, 50, 50, 50), (50, 50, 50, 50, 50), (55, 55, 55), (55, 55, 55, 55), (55, 55, 55, 55, 55), (60, 60, 60), (60, 60, 60, 60), (60, 60, 60, 60, 60)]


In [49]:
start = time.time()
nMin = nRand
nMax = nRand
nStep = 500
vect_train_error = np.zeros(int((nMax-nMin)/nStep)+1)
vect_test_error = np.zeros(int((nMax-nMin)/nStep)+1)
i = 0
for n in range(nMin,nMax+1,nStep):
    index = sample(range(0,nRand),n)
    X=np.zeros((n,len(ArrParams[0])))
    for k in range(len(ArrParams[0])):
        X[:,k]=[ArrParams[j][k] for j in index]
    Y=[iv[j] for j in index]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    # ANN FIT
    vect_train_error[i]=1000
    vect_test_error[i]=1000
    for current_archi in hidden_layers_sizes_vect:
        ann = sknn.MLPRegressor(hidden_layer_sizes=current_archi)
        ann.fit(X_train,Y_train)
        if (np.mean(abs(ann.predict(X_train)-Y_train))*100<vect_train_error[i]):
            vect_train_error[i]=np.mean(abs(ann.predict(X_train)-Y_train))*100
            vect_test_error[i]=np.mean(abs(ann.predict(X_test)-Y_test))*100
    i+=1
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')

Total computation time = 281.95s


In [50]:
print(vect_train_error)
print(vect_test_error)
#plt.plot(range(nMin,nMax+1,nStep),vect_train_error,color='b')
#plt.plot(range(nMin,nMax+1,nStep),vect_test_error, color='r')
#plt.show()

[0.42243789]
[0.4285705]


In [51]:
nTest = 3000
X_test = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nTest)
X_testAfterScale = scaler.transform(X_test)
iv = np.zeros(nTest)
err = np.zeros(nTest)
prediction = ann.predict(X_testAfterScale)
for i in range(nTest):
    HestonModel.setParams([X_test[i,k] for k in range(len(MinParamsValues))])
    iv[i] = EuropeanOption.impliedVolatility(EuropeanOption.NPV(),BSProcess,1.0e-4,200,1.0e-8,10)
    err[i]=abs(iv[i]-prediction[i])
stats.describe(err)

DescribeResult(nobs=3000, minmax=(3.6815891468755524e-07, 0.060570118918586824), mean=0.0047184862438876644, variance=1.4214250562972097e-05, skewness=2.378469056236739, kurtosis=19.338542371363665)

8.426932725704573